#  다차항 회귀분석 클래스 (polynomial regression)

앞으로의 설명을 위해 편의를 위해 다차항 회귀분석을 클래스화 시켜보았다. 
fit, score, plot 부분을 메소드로 변환함으로서 좀더 편의성을 갖는다.


In [ ]:
from sklearn import linear_model
import numpy as np 
import matplotlib.pyplot as plt

class polyRegression :
    
    def createPx(self, x):
        ax = x.reshape(len(x), 1)
        ax = np.c_[x]
        for i in range(1,self.n) :
            ax = np.c_[ax, ax[:,-1] * x]            
        return ax     
            
    def __init__(self, x, y, n) : # 생성자 
        self.x , self.y, self.n = x, y, n
        self.ax = self.createPx(x)
        self.regr = linear_model.LinearRegression() 
        self.regr.fit(self.ax,y)    
    
    def score(self, x=[], y=[] ) :
        if not len(x): 
            x = self.x
        if not len(y): 
            y = self.y
        ax = self.createPx(x)
        
        return self.regr.score(ax, y)
    
    def plot(self, x=[], y=[], mark = '') :
        if not len(x): 
            x = self.x
        if not len(y): 
            y = self.y
        ax = self.createPx(x)
        plt.plot(x,y,'.')   
        ep = self.regr.coef_[0] * ax[:,0]
        for i in range(1,self.n) :        
            ep += ax[:,i] * self.regr.coef_[i] 
        ep += self.regr.intercept_    
        plt.plot(x,ep )


<hr style="height:3px">

#  과적합과 교차검증

## 과적합

다음의 간단한 노이즈가 있는 모델을 보자 


In [ ]:
import numpy as np
import matplotlib.pyplot as plt 
n = 10
x = np.linspace(0,10,n)
y = x + np.random.normal(0,1,n)
plt.plot(x,y,'.')
plt.show()

이제 이 데이터를 위에서 정의한 다차항 회귀분석 클래스에 적용해보자

In [ ]:
pregr = polyRegression(x,y,6)
print("R2:", pregr.score())
pregr.plot()

아마 대부분 0.99 이상의 결정계수를 가진 모델을 생성했을 것이다. 하지만 위 데이터는 x = y 라는 모델에 노이즈가 낀 데이터일 뿐이다. 이제 같은 방식으로 새로운 데이터를 만들어 테스트해보자. 



In [ ]:
n = 10
x2 = np.linspace(0,10,n)
y2 = x2 + np.random.normal(0,1,n)
plt.plot(x2,y2,'.')
plt.show()

이번엔 아까 생성한 모델로 이번 데이터를 예측해보자 

In [ ]:
pregr.plot(x2,y2)
print("R2:", pregr.score(x2,y2))

생각보다 적중도가 훨신 떨어진 것을 볼 수 있다.  이것은 다차항 회귀분석이 노이즈까지 최적화시킴으로 발생하는 문제이다. 이럴댄 차라리 1차항으로 모델을 생성하는게 더 나을것이다. 

In [ ]:
pregr = polyRegression(x,y,1)
pregr.score(x2,y2)
pregr.plot(x2,y2)
print(pregr.regr.coef_)

이것은 처음 다항모델보다 단항 모델이 더 최적화되어있기때문에 발생하는 문제이다. 학습데이터가 풍부하면 이런 문제가 줄어들지만 실제 현실에선 충분한 학습 예제를 얻기 힘든 경우가 많기 때문에 이런 문제가 더 자주 발생한다. 이런 경우를 과적합이라고 표현한다. 

## 데이터의 검증

결과적으로 어떤 데이터가 있을때 학습데이터의 적합도로 모델의 성능을 판단하는 것은 좋지 안다. 이를 위해서는 학습데이터의 일부를 검증용 데이터로 변환하는 것으로 이 문제를 해결할 수 있다. 

In [ ]:
n = 20
x = np.linspace(0,10,n)
y = x*x + np.random.normal(0,1,n)

trainx,testx = x[:15], x[15:]
trainy,testy = y[:15], y[15:]

pregr = polyRegression(trainx, trainy, 1)
pregr.plot()
pregr.plot(testx, testy)
print("R2:", pregr.score(testx,testy))

대부분 높은 정확도를 지녔을거라 생각하지만 생각보다 낮은 정확도를 보이는 사람도 많을 것이다. 이는 데이터의 영역을 전, 후로 나눠서 발생한 문제이다. 이를 좀더 정확하게 하려면 다음과 같이 데이터를 섞은 후에 나눠주는 것이 더 좋다. 

- np.random.shuffle(x) : array x 의 순서를 섞어준다. 
- np.sort(x) : array x 의 순서를 정렬해준다. 


In [ ]:
n = 20
x = np.linspace(0,10,n)
np.random.shuffle(x)

trainx,testx = x[:15], x[15:]
trainx = np.sort(trainx)
testx = np.sort(testx)
trainy = trainx * trainx + np.random.normal(0,1,15)
testy = testx * testx + np.random.normal(0,1,5)

plt.plot(trainx,trainy,'.')
plt.plot(testx,testy, '.')
plt.show()
#y = x + np.random.normal(0,1,n)


이제 위의 섞여진 트레이닝데이터로 회귀분석을 하고 테스트 데이터로 검증을 해보자

In [ ]:
pregr = polyRegression(trainx, trainy, 1)
pregr.plot()
pregr.plot(testx, testy)
plt.show()
print("R2:", pregr.score(testx,testy))

편중성이 많이 사라진 걸 알 수 있을것이다. 

### <font color = 'red'> 연습문제 2-7 : 위의 코드를 20번 돌린 후에 결정계수의 분포를 히스토그램으로 표현해보자 </font>

- [참고] 배열 arr에 값을 추가할때 : 배열.append( 값 )

In [ ]:
# 연습문제 2-7의 코드를 여기에 작성하세요





## k-fold 교차 검증

위의 연습문제를 실행해보면 생각보다 편차가 크고 순간적으로 성능이 확 떨어지는 경우를 볼 수 있을 것이다. 비록 셔플을 이용해 조금 편중을 약화시켰지만 랜덤으로 나누어진 트레이닝과 테스트셋이 편중이 클 확률은 언제라도 존재하다. 그러므로 한번의 실험으로 이 모델이 우수하다는 것을 입증하는 것은 매우 위험하며 이런 문제를 최소화 시키기 위해 오래전부터 k-fold 교차검증 (k-fold cross validation) 을 사용하였다. 


k-fold 교차검증은 데이터를 k 개로 나눈 후에 하나의 그룹을 테스트 데이터로 나머지를 트레이닝 데이터로 선정해 학습을 수행 후에 정확도를 측정하는 과정을 각각 k개의 그룹에 대해 수행하는 것이다.  

예를 들어 100개의 데이터를 5 ( k=5 ) 로 교차검증시 다음과 같이 데이터를 추출한다. 

- arr[0:20] -> test data , arr[20:100] -> training data 
- arr[20:40] -> test data , arr[0:20] + arr[40:100] -> training data 

- :

- arr[80:100] -> test data, arr[0:80] -> training data 

<img src = "https://cdn-images-1.medium.com/max/1600/1*rgba1BIOUys7wQcXcL4U5A.png"  align = left>
출처: https://cdn-images-1.medium.com/max/1600/1*rgba1BIOUys7wQcXcL4U5A.png


## KFold

물론 이런 교차검증을 위한 데이터 생성을 직접 작성하는 것도 크게 어려운 일은 아닐것이다. 하지만 그런 작은 귀찮음조차 방지해 주는 파이썬답게 sklearn 에는 교차검증 데이터 생성용 클래스를 제공한다. 

In [ ]:
from sklearn.model_selection import KFold 

x = np.linspace(0,10,20)
y = x * x 
kf = KFold(n_splits = 4, shuffle = True)
for tr_i, tt_i in kf.split(x):
    print(tr_i,tt_i)

위를 실행하면 인덱스가 랜덤하게 15 : 5 로 분류하되 각 테스트셋이 겹치지 안게 생성될 것이다. KFold 에서 shuffle=true 는 순서를 섞는다는 뜻이다. 다만 이것은 어디까지나 인덱스이며 실제로 x 를 얻어내기 위해서는 x[tr_i], x[tt_i] 로 x 의 트레이닝 셋과 테스트 셋을 얻을 수 있다. 

이것을 이용해 학습을 수행할때는 다음과 같이 수행한다. 

In [ ]:
from sklearn.model_selection import KFold , cross_val_score
from sklearn import linear_model

x = np.linspace(0,10,20)
y = x * x 
kf = KFold(n_splits = 4, shuffle = True)

regr = linear_model.LinearRegression()
for tr_i, tt_i in kf.split(x):
    tr_x, tr_y = x[tr_i], y[tr_i]
    tt_x, tt_y = x[tt_i], y[tt_i]
    
    tr_x = tr_x.reshape(len(tr_x), 1)
    tt_x = tt_x.reshape(len(tt_x), 1)
        
    regr.fit(tr_x, tr_y)
    print(regr.score(tt_x, tt_y))
    

## cross_val_score

작동을 시켜보면 4번의 시행중에 어느정도 편차가 존재함을 알 수 있을 것이다. 하지만 이를 평균 낼 경우 모델의 어느정도 정확도를 낼 수 있다. 심지어는 이조차도 귀찮은 사람들을 위해 cross_val_score 라는 함수로 이를 한번에 수행해준다. 

- cross_val_score( 모델, x, y, cv=Kfold 클래스) 


In [ ]:
from sklearn.model_selection import cross_val_score
x = np.linspace(0,10,20)
y = x * x 

x  = x.reshape(len(x),1)
kf = KFold(n_splits = 4, shuffle = True)
regr = linear_model.LinearRegression()
scores = cross_val_score(regr, x, y, cv=kf)

print(scores)


### <font color = 'red'> 연습문제 2-8 : k-fold 교차검증 반복측정 
    
위의 4번의 시행에 평균값을 20번 반복하여 그 분포를 그려보자 

- [참고] 평균값 구하기 : np.average(배열)



In [ ]:
# 연습문제 2-8의 코드를 작성하고 실행ㅏ자


위에 다항 회귀분석 클래스인 polyRegression 에도 k-fold 교차 검증 기능이 있다면 더 편리할 것이다. 이를 위해 위의 클래스에 cross_validate 메소드를 추가해보도록 하겠다 


In [ ]:
from sklearn import linear_model
from sklearn.model_selection import KFold, cross_val_score

import numpy as np 
import matplotlib.pyplot as plt

class polyRegression :
    
    def createPx(self, x):
        ax = x.reshape(len(x), 1)
        ax = np.c_[x]
        for i in range(1,self.n) :
            ax = np.c_[ax, ax[:,-1] * x]            
        return ax     
            
    def __init__(self, x, y, n) : # 생성자 
        self.x , self.y, self.n = x, y, n
        self.ax = self.createPx(x)
        self.regr = linear_model.LinearRegression() 
        self.regr.fit(self.ax,y)    
    
    def score(self, x=[], y=[] ) :
        if not len(x): 
            x = self.x
        if not len(y): 
            y = self.y
        ax = self.createPx(x)
        
        return self.regr.score(ax, y)
    
    def plot(self, x=[], y=[], mark = '') :
        if not len(x): 
            x = self.x
        if not len(y): 
            y = self.y
        ax = self.createPx(x)
        plt.plot(x,y,'.')   
        ep = self.regr.coef_[0] * ax[:,0]
        for i in range(1,self.n) :        
            ep += ax[:,i] * self.regr.coef_[i] 
        ep += self.regr.intercept_    
        plt.plot(x,ep )
    
    def crossValidate(self, k) :
        kf = KFold(n_splits = k, shuffle = True)
        regr = linear_model.LinearRegression() 
        return cross_val_score( regr, self.ax, self.y, cv=kf)                
    

이제 이걸 이용해 간단한 사인곡선을 6차항 회귀분석으로 cross_validation을 해보도록 하겠다. 

In [ ]:
x = np.linspace(0,np.pi*2, 20)
y = np.sin(x) + np.random.normal(0,0.1,20)
plt.plot(x,y,'.')
plt.show()
pr = polyRegression(x, y, 6)
pr.crossValidate(4)


<hr style="height:3px">

#  Ridge 회귀분석


앞에서는 과적합을 방지하기 위해 교차검증을 통해 모델의 과적합 여부를 분석하는 방법을 살펴보았다. 하지만 이는 어디까지나 생성된 모델의 적합성과 과적합 여부를 판단하는 방법일 뿐이다. 처음부터 근본적으로 과적합을 방지하는 방법은 없을까?

다항분석의 경우 데이터의 복잡도에 비해 조금 낮은 다차항을 선택하면 과적합을 방지하는 효과가 있지만 매번 적절한 다차항의 숫자를 찾는 것은 고통스러운 일임이 틀림 없다.  이러한 Linear Regression 의 과적합 문제를 보완하기 위한 방법이 1970년 개발되었다. 

선형회귀분석(linear regression) 은 오차가 최소화되는 선형모델의 가중치를 찾는 과정이다. 그것을 다음과 같이 표현하기도 한다. 




$$ w = \text{arg}\min_w \left( \sum_{i=1}^N e_i^2  \right) $$


$e_i$ 모델의 예측값과 실제 데이터의 차이 .. 즉 오차를 의미한다. 즉 오차의 제곱을 최소화하는 것이 선형회귀분석 방법이며 결과적으로 x 의 차수가 크고 트레이닝 데이터의 숫자가 적을수록 거의 모든 점을 통과하는 곡선을 생성하며 결정계수는 1에 수렴한다.  

수학자들은 생성된 모델의 변수의 가중치 ( coef_ 값 ) 이 클수록 더 큰 곡률을 가지며 과적합이 발생한다는 사실에 착안하였다. 그리하여 오차만이 아니라 가중치값들의 크기도 고려하여 모델을 생성하는 방법을 제안하였다. 이는 다음과 같이 

결과적으로 이것이 과적합을 만드는 원인으로 보고 여기에 한가지 기준을 더 추가한 것이 Ridge 회귀분석이다. 이는 다음과 같이 표기된다. 

$$w = \text{arg}\min_w \left( \sum_{i=1}^N e_i^2 + \lambda \sum_{j=1}^M w_j^2 \right)$$

위의 식에서 $w_j^2$는 각 가중치 값들의 제곱을 의미한다. 결국 가중치의 제곱의 합도 최소값 생성 기준으로 추가된것이다. 다만 여기에 여기에 $\lambda$ 가 추가되었는데 이는 가중치값과 오차값의 비중을 조정하는 역할을 한다. 만일 $\lambda$ 가 0이라면 가중치는 영향을 주지 못하기 때문에 선형회귀 분석과 같은 역할을 한다. $\lambda$ 가 만일 엄청 크다면 오차보다 w를 최소화 하는 것을 더 중시할 것이다. 가중치의 합이 가장 적은 모델은 모든 가중치가 0 인 모델..  $y = 0 * x^n + 0 * x^n-1 + ... + C $ 형태가 되어서 사실상 $y = C$ 형태의 수평선이 됩니다. 하지만 $\lambda$ 값이 적당하몀ㄴ 이 람다는 모델의 과적합을 막고 더 굴곡이 적은 모델을 생성하는 역할을 합니다. 

사용법은 $\lambda$ 값이 필요하다는 점을 제외하면 LinearRegression과 똑같습니다. 



In [ ]:
from sklearn import linear_model

n = 10
x = np.linspace(0,10,n)
y = x + np.random.normal(0,1,n)
ridge = linear_model.Ridge( alpha = 1.0 )

rx = x.reshape(len(x), 1)
ridge.fit(rx,y)

plt.plot(x,y,'.')
plt.plot(x, ridge.coef_ * x + ridge.intercept_)
plt.show()

이제 위의 polyRegression 을 확장하여 Ridge 모델까지 활용할 있도록 해보겠다. 

In [ ]:
from sklearn import linear_model
from sklearn.model_selection import KFold, cross_val_score

import numpy as np 
import matplotlib.pyplot as plt

class polyRegression :
    
    def createPx(self, x):
        ax = x.reshape(len(x), 1)
        ax = np.c_[x]
        for i in range(1,self.n) :
            ax = np.c_[ax, ax[:,-1] * x]            
        return ax     
            
    def __init__(self, x, y, n, model = linear_model.LinearRegression() ) : # 생성자 
        self.x , self.y, self.n = x, y, n
        self.ax = self.createPx(x)
        self.regr = model
        self.regr.fit(self.ax,y)    
    
    def score(self, x=[], y=[] ) :
        if not len(x): 
            x = self.x
        if not len(y): 
            y = self.y
        ax = self.createPx(x)
        
        return self.regr.score(ax, y)
    
    def plot(self, x=[], y=[], mark = '') :
        if not len(x): 
            x = self.x
        if not len(y): 
            y = self.y
        ax = self.createPx(x)
        plt.plot(x,y,'.')   
        ep = self.regr.coef_[0] * ax[:,0]
        for i in range(1,self.n) :        
            ep += ax[:,i] * self.regr.coef_[i] 
        ep += self.regr.intercept_    
        plt.plot(x,ep )
    
    def crossValidate(self, k) :
        kf = KFold(n_splits = k, shuffle = True)
        regr = linear_model.LinearRegression() 
        return cross_val_score( regr, self.ax, self.y, cv=kf)         

이제 ridge 모델을 사용할때는 다음과 같이 사용하면 된다. 

In [ ]:
x = np.linspace(0,np.pi*2, 20)
y = np.sin(x) + np.random.normal(0,0.1,20)

ridge = linear_model.Ridge( alpha = 1.0 )
rpr = polyRegression(x, y, 6, ridge)
rpr.plot(x,y)
print("R2:", rpr.score(x,y))
plt.show()

이제 이를 linear regression 과 비교해보자. 

In [ ]:
lpr = polyRegression(x, y, 6)
lpr.plot(x,y)
print("R2:", lpr.score(x,y))
plt.show()

linear regression 에 비해 결정계수가 낮은걸 볼 수 있다. 하지만 만일 알파값을 0으로 둘 경우는 linear regression과 같은 결과를 보인다. 

In [ ]:
ridge = linear_model.Ridge( alpha = 0 )
rpr = polyRegression(x, y, 6, ridge)
rpr.plot(x,y)
print("R2:", rpr.score(x,y))
plt.show()

꺼꾸로 알파값을 아주 높여보도록 하겠다. 

In [ ]:
ridge = linear_model.Ridge( alpha = 10.0 )
rpr = polyRegression(x, y, 6, ridge)
rpr.plot(x,y)
print("R2:", rpr.score(x,y))
plt.show()

이번엔 훨씬더 유연해진 곡선을 볼 수 있을것이다. 이런 일반화 과정을 통해 ridge 회귀분석은 데이터의 지나친 과적합을 막게 되며 다차항 함수의 단점도 막아줄 수 있다. 

### <font color = 'red'> 연습문제 2-9 : 직선데이터 교차검증 비교 
    
다음 데이터셋에 6차항의 linear_regression 과 ridge 에 각각  k=4 교차검증으로 둘의 성능을 비교해보자 


In [ ]:
from sklearn.model_selection import KFold 
n = 20
x = np.linspace(0,10,n)
y = x + np.random.normal(0,1,n)

plt.plot(x,y,'.')
    

In [ ]:
# 연습문제 2-9 를 여기에 작성하세요